In [1]:
import ROOT as R
import re
import argparse
from array import array
import os
R.EnableImplicitMT()

Welcome to JupyROOT 6.26/10


### plot functions

In [2]:
def add_lumi(year):
    lowX=0.55
    lowY=0.835
    lumi  = R.TPaveText(lowX, lowY+0.06, lowX+0.30, lowY+0.16, "NDC")
    lumi.SetBorderSize(   0 )
    lumi.SetFillStyle(    0 )
    lumi.SetTextAlign(   12 )
    lumi.SetTextColor(    1 )
    lumi.SetTextSize(0.06)
    lumi.SetTextFont (   42 )
    if (year=="2018"): lumi.AddText("2018, 60 fb^{-1} (13 TeV)")
    if (year=="2017"): lumi.AddText("2017, 41 fb^{-1} (13 TeV)")
    if (year=="2016"): lumi.AddText("2016, 36 fb^{-1} (13 TeV)")
    return lumi

def add_CMS():
    lowX=0.11
    lowY=0.835
    lumi  = R.TPaveText(lowX, lowY+0.06, lowX+0.15, lowY+0.16, "NDC")
    lumi.SetTextFont(61)
    lumi.SetTextSize(0.08)
    lumi.SetBorderSize(   0 )
    lumi.SetFillStyle(    0 )
    lumi.SetTextAlign(   12 )
    lumi.SetTextColor(    1 )
    lumi.AddText("CMS")
    return lumi

def add_Preliminary():
    lowX=0.25
    lowY=0.835
    lumi  = R.TPaveText(lowX, lowY+0.06, lowX+0.15, lowY+0.16, "NDC")
    lumi.SetTextFont(52)
    lumi.SetTextSize(0.06)
    lumi.SetBorderSize(   0 )
    lumi.SetFillStyle(    0 )
    lumi.SetTextAlign(   12 )
    lumi.SetTextColor(    1 )
    lumi.AddText("Preliminary")
    return lumi

def make_legend():
        output = R.TLegend(0.5, 0.65, 0.92, 0.86, "", "brNDC")
        output.SetNColumns(2)
        output.SetLineWidth(0)
        output.SetLineStyle(0)
        output.SetFillStyle(0)
        output.SetBorderSize(0)
        output.SetTextFont(62)
        return output

def make_legend2():
        output = R.TLegend(0.45, 0.6, 0.92, 0.86, "", "brNDC")
        output.SetNColumns(2)
        output.SetLineWidth(0)
        output.SetLineStyle(0)
        output.SetFillStyle(0)
        output.SetBorderSize(0)
        output.SetTextFont(62)
        return output

### basic IO

In [10]:
path = "/gwpool/users/lzhang/private/bbtt/CMS-HHbbtt-boosted-BigN/" + "AK8based_Out_condor/"

process_list = ['SMHH', 'DY+Jets50To100', 'DY+Jets100To250', 'DY+Jets250To400', 'DY+Jets400To650', 'DY+Jets650ToInf','VBFH', 'ggFH','TTbarHad', 'TTbarSemi', 'TTbarDiLep']
sample_list = ['SMHH', 'DY+Jets50To100', 'DY+Jets100To250', 'DY+Jets250To400', 'DY+Jets400To650', 'DY+Jets650ToInf','VBFH', 'ggFH','TTbarHad', 'TTbarSemi', 'TTbarDiLep']
Sig_list = ['SMHH']
SH_list = ['VBFH', 'ggFH']
DY_list = ['DY+Jets50To100', 'DY+Jets100To250', 'DY+Jets250To400', 'DY+Jets400To650', 'DY+Jets650ToInf']
TT_list = ['TTbarHad', 'TTbarSemi', 'TTbarDiLep']

fileset = {}
fileset['SMHH'] = []
fileset['VBFH'] = []
fileset['ggFH'] = []
fileset['DY+Jets50To100'] = []
fileset['DY+Jets100To250'] = []
fileset['DY+Jets250To400'] = []
fileset['DY+Jets400To650'] = []
fileset['DY+Jets650ToInf'] = []
fileset['TTbarHad'] = []
fileset['TTbarSemi'] = []
fileset['TTbarDiLep'] = []
# for process in process_list:
for process in sample_list:
    for file in os.listdir(path+process):
        if file.endswith(".root"):
            fileset[process].append(path+'/'+process+'/'+file)

In [11]:
df_dict = {}
histo_dict = {}

In [12]:
interested_variables = {"ak8jets_SoftDropMass","true_weight","ak8jets_Pt","ak8jets_Eta","ak8jets_Mass","ak8jets_probHtt","match_gen_tau","match_gen_hav","bParticleNetTauAK8JetTags_probHtt"}
for sample in sample_list:
    df_dict[sample] = R.RDataFrame("ak8tree", fileset[sample], interested_variables)
    df_dict[sample+'_T1'] = df_dict[sample].Filter("abs(ak8jets_Eta)<2.5").Filter("ak8jets_Mass>30").Filter("ak8jets_Pt>250").Filter("match_gen_tau==2")
    df_dict[sample+'_T2'] = df_dict[sample].Filter("abs(ak8jets_Eta)<2.5").Filter("ak8jets_Mass>30").Filter("ak8jets_Pt>250").Filter("match_gen_tau==1")
    df_dict[sample+'_T3'] = df_dict[sample].Filter("abs(ak8jets_Eta)<2.5").Filter("ak8jets_Mass>30").Filter("ak8jets_Pt>250").Filter("match_gen_tau==0").Filter("match_gen_hav>0")
    df_dict[sample+'_T4'] = df_dict[sample].Filter("abs(ak8jets_Eta)<2.5").Filter("ak8jets_Mass>30").Filter("ak8jets_Pt>250").Filter("match_gen_tau==0").Filter("match_gen_hav==0")
    

In [13]:
for sample in Sig_list:
    for jet_type in ['T1', 'T2', 'T3', 'T4']:
        print(f'{sample}_{jet_type}')
        histo_dict[f'{sample}_{jet_type}_bParticleNetTauAK8JetTags_probHtt'] = \
        df_dict[f'{sample}_{jet_type}'].Histo1D((" ", " ", 100, 0, 1), "bParticleNetTauAK8JetTags_probHtt", "true_weight").GetPtr()
        

SMHH_T1
SMHH_T2
SMHH_T3
SMHH_T4


In [14]:
for sample in SH_list:
    for jet_type in ['T1', 'T2', 'T3', 'T4']:
        print(f'{sample}_{jet_type}')
        histo_dict[f'{sample}_{jet_type}_bParticleNetTauAK8JetTags_probHtt'] = \
        df_dict[f'{sample}_{jet_type}'].Histo1D((" ", " ", 100, 0, 1), "bParticleNetTauAK8JetTags_probHtt", "true_weight").GetPtr()
        

VBFH_T1
VBFH_T2
VBFH_T3
VBFH_T4
ggFH_T1
ggFH_T2
ggFH_T3
ggFH_T4


In [ ]:
for sample in DY_list:
    for jet_type in ['T1', 'T2', 'T3', 'T4']:
        print(f'{sample}_{jet_type}')
        histo_dict[f'{sample}_{jet_type}_bParticleNetTauAK8JetTags_probHtt'] = \
        df_dict[f'{sample}_{jet_type}'].Histo1D((" ", " ", 100, 0, 1), "bParticleNetTauAK8JetTags_probHtt", "true_weight").GetPtr()
        

In [16]:
for sample in TT_list:
    for jet_type in ['T1', 'T2', 'T3', 'T4']:
        print(f'{sample}_{jet_type}')
        histo_dict[f'{sample}_{jet_type}_bParticleNetTauAK8JetTags_probHtt'] = \
        df_dict[f'{sample}_{jet_type}'].Histo1D((" ", " ", 100, 0, 1), "bParticleNetTauAK8JetTags_probHtt", "true_weight").GetPtr()
        

TTbarHad_T1
TTbarHad_T2
TTbarHad_T3
TTbarHad_T4
TTbarSemi_T1
TTbarSemi_T2
TTbarSemi_T3
TTbarSemi_T4
TTbarDiLep_T1
TTbarDiLep_T2
TTbarDiLep_T3
TTbarDiLep_T4


IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out


In [17]:
histo_dict['T1'] = histo_dict['SMHH_T1_bParticleNetTauAK8JetTags_probHtt'].Scale(0.01)
histo_dict['T2'] = histo_dict['SMHH_T2_bParticleNetTauAK8JetTags_probHtt'].Scale(0.01)
histo_dict['T3'] = histo_dict['SMHH_T3_bParticleNetTauAK8JetTags_probHtt'].Scale(0.01)
histo_dict['T4'] = histo_dict['SMHH_T4_bParticleNetTauAK8JetTags_probHtt'].Scale(0.01)

In [18]:
for sample in sample_list[1:]:
    for jet_type in ['T1', 'T2', 'T3', 'T4']:
        print(f'{sample}_{jet_type}')
        histo_dict[f'{jet_type}'] = histo_dict[f'{jet_type}'] + histo_dict[f'{sample}_{jet_type}_bParticleNetTauAK8JetTags_probHtt']
        

DY+Jets50To100_T1


NotImplementedError: 

In [ ]:
## Efficiency plot

histo_dict['Eff_T1'] = R.TH1F("Eff_T1", "", 100, 0, 1)
histo_dict['Eff_T2'] = R.TH1F("Eff_T2", "", 100, 0, 1)
histo_dict['Eff_T3'] = R.TH1F("Eff_T3", "", 100, 0, 1)
histo_dict['Eff_T4'] = R.TH1F("Eff_T4", "", 100, 0, 1)

for jet_type in ['T1', 'T2', 'T3', 'T4']:
    for ibin in range(0,100):
        eff = histo_dict[f'{jet_type}'].Integral(0, ibin+1)/histo_dict[f'{jet_type}'].Integral()
        histo_dict[f'Eff_T{jet_type}'].SetBinContent(ibin+1, eff)